IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import array 
import scipy.signal as signal
#Processing libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import seaborn as sns
from scipy.stats import zscore

# Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA

# Testing libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, learning_curve
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report, roc_curve, auc

# remove warnings
import warnings
warnings.filterwarnings('ignore')

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva


ARRAY CHE CONTERRA' I DATAFRAME DI CIASCUN VOLO (PER UN TOTALE DI 18)

In [3]:
data_path = os.path.join(path_file,'*.csv') #lista di tutti gli elementi di estensione csv nella cartella path
csv_list = glob.glob(data_path) #converte data path in un output Unix-like (ls) (*.csv -> lista di elementi con estensione csv)
dataframe_collection = []
df_1 = pd.DataFrame()

# ciclo per scorrere tutti i csv
for csv_file in csv_list: # ciclo che scorre i csv nella cartella path
    df = pd.read_csv(csv_file) 
    #df = df.to_numpy()
    #df = df.to_records(index=False)
    #dataframe_collection.append(df)
    df_1 = df_1.append(df,ignore_index=True)
    #print(df)

# ciclo per scorrere tutti i csv
for csv_file in csv_list: # ciclo che scorre i csv nella cartella path
    df = pd.read_csv(csv_file) 
    #df = df.to_numpy()
    df = df.to_records(index=False)
    dataframe_collection.append(df)

#prova = pd.DataFrame(dataframe_collection)


In [4]:
print(df_1)

      tempo  <>(GyrX)  var(GyrX)  kurt(GyrX)  RMS(GyrX)  freq1(GyrX)  \
0       0.0 -0.032100   0.017081   -0.399284   0.134398         78.0   
1       1.0 -0.032346   0.017054   -0.400010   0.134356        102.0   
2       2.0 -0.032590   0.017026   -0.400594   0.134314         98.0   
3       3.0 -0.032832   0.016999   -0.401113   0.134271        102.0   
4       4.0 -0.033265   0.017017   -0.412325   0.134447        102.0   
...     ...       ...        ...         ...        ...          ...   
1344   62.0 -0.048472   0.006254   -0.082119   0.092673          0.0   
1345   63.0 -0.048289   0.006253   -0.081993   0.092571          0.0   
1346   64.0 -0.048212   0.006240   -0.075446   0.092463          0.0   
1347   65.0 -0.048179   0.006225   -0.068067   0.092367          0.0   
1348   66.0 -0.048194   0.006211   -0.061448   0.092295          0.0   

      freq2(GyrX)  amp1(GyrX)  amp2(GyrX)  <>(GyrY)  ...  amp2(RPM(ESC_5))  \
0            96.0    3.596543    3.414090 -0.014483  ... 

In [5]:
df_1 = df_1.sample(frac=1).reset_index(drop=True)
print(df_1)

      tempo  <>(GyrX)  var(GyrX)  kurt(GyrX)  RMS(GyrX)  freq1(GyrX)  \
0       4.0 -0.062441   0.012834    0.178231   0.129216          0.0   
1      17.0 -0.058863   0.005912    0.047303   0.096749          0.0   
2      70.0 -0.014774   0.017749    3.548264   0.133886          0.0   
3      86.0 -0.052679   0.025397   -0.909265   0.167673          0.0   
4      10.0 -0.140203   0.034049   -0.827969   0.231542        103.0   
...     ...       ...        ...         ...        ...          ...   
1344   76.0 -0.011131   0.057593    1.920927   0.239962         88.0   
1345   14.0 -0.043772   0.019178   -0.871602   0.145055         73.0   
1346   66.0 -0.048194   0.006211   -0.061448   0.092295          0.0   
1347   69.0  0.001150   0.007084   -0.282288   0.084075          0.0   
1348   51.0 -0.129222   0.032372   -0.700008   0.221338          0.0   

      freq2(GyrX)  amp1(GyrX)  amp2(GyrX)  <>(GyrY)  ...  amp2(RPM(ESC_5))  \
0            67.0   11.975022    5.459079  0.024647  ... 

In [6]:
df_1 = df_1.drop(['tempo'], axis=1)
print(df_1)

      <>(GyrX)  var(GyrX)  kurt(GyrX)  RMS(GyrX)  freq1(GyrX)  freq2(GyrX)  \
0    -0.062441   0.012834    0.178231   0.129216          0.0         67.0   
1    -0.058863   0.005912    0.047303   0.096749          0.0         98.0   
2    -0.014774   0.017749    3.548264   0.133886          0.0        116.0   
3    -0.052679   0.025397   -0.909265   0.167673          0.0         98.0   
4    -0.140203   0.034049   -0.827969   0.231542        103.0          0.0   
...        ...        ...         ...        ...          ...          ...   
1344 -0.011131   0.057593    1.920927   0.239962         88.0         87.0   
1345 -0.043772   0.019178   -0.871602   0.145055         73.0         56.0   
1346 -0.048194   0.006211   -0.061448   0.092295          0.0        113.0   
1347  0.001150   0.007084   -0.282288   0.084075          0.0        119.0   
1348 -0.129222   0.032372   -0.700008   0.221338          0.0        129.0   

      amp1(GyrX)  amp2(GyrX)  <>(GyrY)  var(GyrY)  ...  amp2(RP

In [ ]:
'''correlated_features = set()
correlation_matrix = df_1.corr()
print(correlation_matrix)

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

print(len(correlated_features))
print(correlated_features)

df_1 = df_1.drop([col for col in df_1.columns if col in correlated_features],axis=1)
print(df_1)'''

In [ ]:
index_list = [470]
df_test = df_1.query('index in @index_list')
#df_test = pd.DataFrame(df_test)
print(df_test)

In [ ]:
#df_1 = df_1.drop([df_1.index[1330], df_1.index[1331]])
df_1 = df_1.drop([df_1.index[470]])
print(df_1)

In [7]:
guasto_10 = df_1[df_1['Guasto'] == 2]
no_guasto = df_1[df_1['Guasto'] == 0]
guasto_5 = df_1[df_1['Guasto'] == 1]

print("No guasto: ", len(no_guasto), "Guasto 5%: ", len(guasto_5), "Guasto 10%: ", len(guasto_10))

No guasto:  390 Guasto 5%:  418 Guasto 10%:  541


### Bilanciamento del dataset

In [8]:
#bilanciamento del dataset con undersampling rispetto al no_guasto
balanced_d = pd.concat([no_guasto, guasto_10.sample(len(no_guasto)), guasto_5.sample(len(no_guasto))])
x = balanced_d.iloc[:,:-1]
y = balanced_d.iloc[:,-1:]
balanced_d.shape

(1170, 169)

In [ ]:
x = df_1.iloc[:,:-1]
y = df_1.iloc[:,-1:]

In [9]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, f_regression
fvalue_Best = SelectKBest(score_func=f_regression, k=10)
fit = fvalue_Best.fit(x, y)
print(fit)
print(fvalue_Best.get_support())

SelectKBest(score_func=<function f_regression at 0x00000221F9E551B0>)
[False False False False False False False False False  True False  True
  True False False False False False False False False False False False
 False False False False  True False False False False False  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True

In [10]:
features_score = pd.DataFrame(fit.scores_)
features_score = zscore(features_score)
features = pd.DataFrame(x.columns)
feature_score = pd.concat([features,features_score],axis=1)
# Assigning column names
feature_score.columns = ["Input_Features","F_Score"]
print(feature_score.nlargest(15,columns="F_Score"))

        Input_Features   F_Score
152     <>(RPM(ESC_5))  6.640353
155    RMS(RPM(ESC_5))  4.963286
34          kurt(AccY)  3.855262
28         freq1(AccX)  2.579611
12         freq1(GyrY)  2.194645
11           RMS(GyrY)  2.066293
93   freq2(RPM(ESC_1))  2.029870
92   freq1(RPM(ESC_1))  2.023110
154   kurt(RPM(ESC_5))  1.986209
9            var(GyrY)  1.975595
14          amp1(GyrY)  1.767075
104     <>(RPM(ESC_2))  1.661637
161   var(Curr(ESC_5))  1.544159
107    RMS(RPM(ESC_2))  1.407990
78    amp1(RPM(ESC_0))  1.386471


In [11]:
i=0
index_false=[]
for el in list(fvalue_Best.get_support()):
    if not el:
        index_false.append(i)
    i=i+1
print(index_false)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167]


In [12]:
x=x.drop(x.columns[index_false],axis = 1)
print(x)

      var(GyrY)  RMS(GyrY)  freq1(GyrY)  freq1(AccX)  kurt(AccY)  \
0      0.005951   0.080883         63.0        104.0   -0.457875   
1      0.003681   0.068790         97.0        120.0   -0.706177   
4      0.004053   0.064866         71.0        107.0   -0.410029   
8      0.003118   0.057395         95.0        119.0   -0.593661   
9      0.007137   0.088343         93.0        109.0   -0.480917   
...         ...        ...          ...          ...         ...   
371    0.011735   0.126812         95.0        108.0    0.180708   
884    0.005646   0.077599         82.0        119.0    0.139346   
909    0.012405   0.122436        118.0        120.0    0.133475   
893    0.005297   0.073924        128.0        122.0    0.480803   
1032   0.006214   0.079168        108.0         84.0    0.151073   

      freq1(RPM(ESC_1))  freq2(RPM(ESC_1))  <>(RPM(ESC_5))  kurt(RPM(ESC_5))  \
0                  32.0               30.0     4952.402817         -1.239046   
1                  97.0

### Separazione del dataset in training e test

In [13]:
# Random state
rs = 42

# Split the data to check which algorithms learn better (later on we can check )
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=rs)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# look at the shape of the data (many problems can arise from wrong shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(936, 10)
(234, 10)
(936, 1)
(234, 1)


In [ ]:
#from sklearn.feature_selection import SelectKBest, mutual_info_classif
#sel_mutual = SelectKBest(mutual_info_classif, k=6)
#X_train_mutual = sel_mutual.fit_transform(x_train, y_train)
#print(sel_mutual.get_support())

#i=0
#index_false=[]
#for el in list(sel_mutual.get_support()):
#    if not el:
#        index_false.append(i)
#    i=i+1
#print(index_false)

In [ ]:
#x_train=x_train.drop(x_train.columns[index_false],axis = 1)
#x_test=x_test.drop(x_test.columns[index_false],axis = 1)

In [ ]:
x_train

In [14]:
# List of classifiers:
classifiers = [
    LogisticRegression(random_state = rs),
    DecisionTreeClassifier(random_state=rs),
    RandomForestClassifier(n_estimators = 10, random_state=rs),
    GradientBoostingClassifier(random_state= rs, n_estimators=10),
    AdaBoostClassifier(n_estimators=10, random_state= rs),
    LinearDiscriminantAnalysis(),
    SVC(probability=True),
    GaussianNB(),
    KNeighborsClassifier(n_neighbors=10)
]

# List of results that will occure:
clf_name = [] # names of the classifiers
model_results = pd.DataFrame.copy(y_test) #resulting of prediction from the models

kfold = StratifiedKFold(n_splits=5) #cross-validation
cv_results = [] # scores from cross validation
cv_acc = [] # mean accuracy from cross validation, need to maximize
cv_std = [] # standard deviation from cross validation, need to minimise

cnfm = [] #confusion matrix
clr = [] #classification report
roc_auc = [] #roc curve:
roc_tpr = []
roc_fpr = []

### Classificazione

In [15]:
# Training the algorithms and results
for clf in classifiers:
    name = clf.__class__.__name__
    clf_name.append(name)
    
    #fitting and predictions
    model = clf.fit(x_train, y_train.values.ravel())
    y_pred = model.predict(x_test)
    model_results[name] = y_pred
    
    #accuracy and log loss
    cv_results.append(cross_val_score(clf, x_train, y_train.values.ravel(), scoring = "accuracy",cv = kfold))
    acc = round(accuracy_score(y_test.values.ravel(), y_pred), 2) #need to maximize
    train_pred = clf.predict_proba(x_test)
    print(f'Accuracy: {acc} \t ---> {name} ')
    
    #confusion matrix, clasification report, roc curve
    cnfm.append(confusion_matrix(y_test.values.ravel(), y_pred))
    clr.append(classification_report(y_test.values.ravel(), y_pred))
    fpr, tpr, thresholds = roc_curve(y_pred, y_test.values.ravel(), pos_label=1)
    roc_auc.append(auc(fpr, tpr))
    roc_tpr.append(tpr)
    roc_fpr.append(fpr)
    

for i in cv_results:
    cv_acc.append(i.mean())
    cv_std.append(i.std())

Accuracy: 0.81 	 ---> LogisticRegression 
Accuracy: 1.0 	 ---> DecisionTreeClassifier 
Accuracy: 1.0 	 ---> RandomForestClassifier 
Accuracy: 1.0 	 ---> GradientBoostingClassifier 
Accuracy: 0.94 	 ---> AdaBoostClassifier 
Accuracy: 0.99 	 ---> LinearDiscriminantAnalysis 
Accuracy: 0.61 	 ---> SVC 
Accuracy: 0.98 	 ---> GaussianNB 
Accuracy: 0.99 	 ---> KNeighborsClassifier 


### Matrici di confusione

In [ ]:
import seaborn as sns
# Confusion matrixes (not-normalized confusion matrix)
plt.figure(figsize=(20,15))
sns.set(font_scale=1.4)
for i in range(len(classifiers)):
    plt.subplot(3,3,i+1) #adjust this acourding to the number of algorithms
    sns.heatmap(cnfm[i], annot=True, fmt="d",cmap="Reds")
    plt.subplots_adjust(hspace = 0.5)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(clf_name[i])

In [ ]:
#Classification reports
for i in range(len(classifiers)):
    print (f"{clf_name[i]} Classification Report:" )
    print (clr[i])

In [ ]:
#clf = SVC(probability=True).fit(x_train, y_train)
clf = GaussianNB().fit(x_train, y_train)
scores = cross_val_score(clf, x_test, y_test, cv=5)
print(scores)

TESTING DEL MODELLO

In [ ]:
# Drop values to avoid multicolinearity
df_test = df_test.drop(['Guasto'], axis=1)

In [ ]:
df_test=df_test.drop(df_test.columns[index_false],axis = 1)
print(df_test)

In [ ]:
guasto_pred = pd.Series(clf.predict(df_test), name='Guasto_test')
guasto_pred.value_counts()